# Linearization

Linearization is an approach developed by [Yandex](https://www.researchgate.net/publication/322969314_Consistent_Transformation_of_Ratio_Metrics_for_Efficient_Online_Controlled_Experiments) that allows you to efficiently work with your ratio data.  
This example notebook will show you how to apply linearization in ABacus.

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=[FutureWarning, DeprecationWarning])

import sys
import logging
import pandas as pd
import numpy as np

from abacus.auto_ab.abtest import ABTest, ABTestParams
from abacus.auto_ab.params import DataParams, HypothesisParams

logging.basicConfig(level = logging.INFO)

%load_ext autoreload
%autoreload 2

In [ ]:
df = pd.read_csv('./data/ab_data_height.csv')
df.head()

In [ ]:
data_params = DataParams(
    id_col='id', 
    group_col='groups', 
    control_name='A',
    treatment_name='B',
    numerator='numerator', 
    denominator='denominator', 
    covariate='height_prev',
    is_grouped=False
)

hypothesis_params = HypothesisParams(
    alpha=0.05, 
    beta=0.2, 
    alternative='greater', 
    metric_type='ratio', 
    metric_name='mean', 
    metric=np.mean, 
)

ab_params = ABTestParams(data_params, hypothesis_params)

In [ ]:
ab_test = ABTest(df, ab_params)
ab_test_1 = ab_test.linearization()

In [ ]:
ab_test_1.params.hypothesis_params.metric_type

As can be seen, after linearization metric type switched to 'continuous' and metric now is continuous.

We can observe new column `target_linearized` in a dataset:

In [ ]:
ab_test_1.dataset

As we have now continuous metric instead of ratio, we can use any metric transformation, e.g. CUPED:

In [ ]:
ab_test_2 = ab_test_1.cuped()
ab_test_3 = ab_test_2.test_welch()